In [1]:
import pandas as pd
from sklearn import preprocessing
from neupy import environment
from sklearn.model_selection import train_test_split
from neupy import environment
from operator import itemgetter
import numpy as np
from neupy import algorithms, layers, estimators
from sklearn import datasets, grid_search
from sklearn.model_selection import GridSearchCV


def scorer(actual, predicted):
    return estimators.rmse(predicted, actual)

# training_df = pd.read_csv(
#     "/home/pier/Machine_Learning/KE5206NN/regression/data_with_fields_removed/train_70.0_fields_removed.csv")
training_df = pd.read_csv(
    "/Users/pierlim/PycharmProjects/KE5206NN/regression/data_with_fields_removed/train_70.0_updated.csv")
training_X = training_df.loc[:, training_df.columns != " shares"]
training_Y = training_df.loc[:, " shares"]

# test_df = pd.read_csv(
#     "/home/pier/Machine_Learning/KE5206NN/regression/data_with_fields_removed/test_30.0_fields_removed.csv")
test_df = pd.read_csv(
    "/Users/pierlim/PycharmProjects/KE5206NN/regression/data_with_fields_removed/test_30.0_updated.csv")
testing_X = test_df.loc[:, test_df.columns != " shares"]
testing_Y = test_df.loc[:, " shares"]

print("Training shapes")
print(training_X.shape)
print(training_Y.shape)

print("Testing shapes")
print(testing_X.shape)
print(testing_Y.shape)

x_data_scaler = preprocessing.MinMaxScaler()
y_data_scaler = preprocessing.MinMaxScaler()

columns_to_scale = [" n_tokens_title", " n_tokens_content", " num_hrefs", " num_self_hrefs", " num_imgs", " num_videos", " average_token_length", " num_keywords", " kw_avg_min"," kw_avg_max", " kw_avg_avg"," self_reference_avg_sharess"]

x_data_scaler.fit(training_X.loc[:, columns_to_scale])
y_data_scaler.fit(training_Y.reshape(-1, 1))

training_X.loc[:, columns_to_scale] = x_data_scaler.transform(training_X.loc[:, columns_to_scale])
training_Y = y_data_scaler.transform(training_Y.reshape(-1, 1))

testing_X.loc[:, columns_to_scale] = x_data_scaler.transform(testing_X.loc[:, columns_to_scale])
testing_Y = y_data_scaler.transform(testing_Y.reshape(-1, 1))


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Training shapes
(27750, 27)
(27750,)
Testing shapes
(11894, 27)
(11894,)


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:44: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:47: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:50: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [2]:
from sklearn.neural_network import MLPRegressor
from scipy import stats
from sklearn.grid_search import RandomizedSearchCV
from neupy import estimators
import _pickle

with open('/Users/pierlim/PycharmProjects/KE5206NN/regression/regression_models/multi_layer_perceptron.pkl', 'rb') as fid:
    mlp = _pickle.load(fid)

with open('/Users/pierlim/PycharmProjects/KE5206NN/regression/regression_models/grnn.pkl', 'rb') as fid:
    grnn = _pickle.load(fid)




In [3]:

y_mlp_predicted = mlp.predict(testing_X)
print("MLP RMSE = " + str(estimators.rmse(y_mlp_predicted, testing_Y.ravel())))
mlp_mae = estimators.mae(y_mlp_predicted, testing_Y.ravel())
print("MLP MAE = " + str(estimators.mae(y_mlp_predicted, testing_Y.ravel())))
actual_mae = y_data_scaler.inverse_transform(estimators.mae(y_mlp_predicted, testing_Y))
print("MLP MAE (no. of shares) = " + str(actual_mae.squeeze()))

y_grnn_predicted = grnn.predict(testing_X)
y_grnn_predicted = y_grnn_predicted.ravel().transpose()
grnn_mae = estimators.mae(y_grnn_predicted, testing_Y.ravel())
print("GRNN RMSE = " + str(estimators.rmse(y_grnn_predicted, testing_Y.ravel())))
print("GRNN MAE = " + str(estimators.mae(y_grnn_predicted, testing_Y.ravel())))
actual_mae = y_data_scaler.inverse_transform(estimators.mae(y_grnn_predicted, testing_Y))
print("GRNN MAE (no. of shares) = " + str(actual_mae.squeeze()))
print(y_grnn_predicted.shape)
print(y_mlp_predicted.shape)


MLP RMSE = 0.018917390394580104
MLP MAE = 0.004566708750875336
MLP MAE (no. of shares) = 3153.851154895581


GRNN RMSE = 0.01886015925064978
GRNN MAE = 0.0043920011010804665
GRNN MAE (no. of shares) = 3033.233168184853
(11894,)
(11894,)


In [9]:

models = [mlp, grnn]
y_ensemble_predicted = np.zeros(shape=(testing_X.shape[0], 1))
#print(y_ensemble_predicted.shape)
denom = 0

y_grnn_predicted_new = (1.0 / (1 + grnn_mae)) * y_grnn_predicted
print("GRNN 1st 10 :")
print(y_grnn_predicted_new[1:10])
y_mlp_predicted_new = (1.0 / (1+ mlp_mae)) * y_mlp_predicted
print("MLP 1st 10 :")
print(y_mlp_predicted_new[1:10])
y_ensemble_predicted = np.add(y_grnn_predicted_new, y_mlp_predicted_new)
ensemble_mae = estimators.mae(y_ensemble_predicted, testing_Y.ravel())
# for model in models:
#     y_predicted = model.predict(testing_X)
#     mae = estimators.mae(y_predicted, testing_Y.ravel())
#     denom = denom + (1.0 / (1+mae))
#     y_ensemble_predicted = np.add(y_ensemble_predicted, ((1.0 / (1+mae)) * y_predicted))

#print(y_ensemble_predicted.shape)
print("Ensemble 1st 10 :")
print(y_ensemble_predicted[1:10])

GRNN 1st 10 :
[ 0.00334197  0.00408681  0.00319064  0.00379061  0.00311479  0.00697594
  0.00391235  0.00315756  0.00365893]
MLP 1st 10 :
[ 0.00400809  0.00606273  0.00398822  0.00286775  0.00373475  0.00308362
  0.00482881  0.00357838  0.00356908]
Ensemble 1st 10 :
[ 0.00735006  0.01014954  0.00717885  0.00665836  0.00684955  0.01005956
  0.00874116  0.00673594  0.007228  ]


(11894,)
[ 0.00735006  0.01014954  0.00717885  0.00665836  0.00684955  0.01005956
  0.00874116  0.00673594  0.007228  ]


In [7]:
denom = (1.0 / (1+grnn_mae)) + (1.0 / (1+mlp_mae))
print(denom)
ensemble_mae = estimators.mae(y_ensemble_predicted, testing_Y.ravel())
# ensemble_mae = ensemble_mae / denom # disabled this
print("Ensemble MAE = " + str(ensemble_mae))
actual_mae = y_data_scaler.inverse_transform(ensemble_mae)
print("Ensemble MAE (no. of shares) = " + str(actual_mae.squeeze()))

1.9910812554957453
Ensemble MAE = 0.007703344551670389
Ensemble MAE (no. of shares) = 5319.381375128685
